Regression linéaire avec toutes les variables:

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

Data_X = pd.read_csv('Data_X.csv')
Data_Y = pd.read_csv('Data_Y.csv')

lin = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(Data_X, Data_Y, test_size=0.1, random_state=42)
lin.fit(X_train, y_train)

lin.score(X_train, y_train)

print("R2 Score", r2_score(y_test, lin.predict(X_test)))
print("MSE(Train)" ,mean_squared_error(y_train, lin.predict(X_train)))

To_save = pd.DataFrame()

To_save['ID'] =  Data_N["ID"]
res = []
for i in range(len(To_save)):
    res.append(lin.predict([Data_N.iloc[i, :]])[0])


Regression de RIDGE avec toutes les variables:

In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge

Data_Y = pd.read_csv('Data_Y.csv')


X_train, X_test, y_train, y_test = train_test_split(Data_X, Data_Y, test_size=0.3, random_state=42)
r=[]
lst = np.linspace(6.681, 1000, 1868).tolist()


for alf in tqdm(lst):
    rid = Ridge(alf).fit(X_train, y_train)
    r2 = r2_score(y_test, rid.predict(X_test))
    r.append({'beta': alf, 'r2': r2})

df = pd.DataFrame(r)
plt.figure(figsize=(6, 2))
plt.subplot(1,2,1)
best_beta = df.loc[df['r2'].idxmax()]['beta']
print("Best beta: ",best_beta)
plt.plot(df['beta'], df['r2'],label = 'r2')
plt.tight_layout()
plt.show()


SyntaxError: invalid decimal literal (1420857375.py, line 3)